In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 11)  #set default figure size
import numpy as np
import os 
import pandas as pd
from ailearn.Swarm import ES
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, DotProduct, ConstantKernel as C
from sklearn.svm import SVC,SVR
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from scipy.optimize import leastsq
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
from sklearn import preprocessing

In [ ]:
## Seperate training and test data
def split(xx, yy):
    zscore = StandardScaler()        
    scaler =MinMaxScaler()   
    xx = zscore.fit_transform(xx)  
    yy = scaler.fit_transform(np.reshape(yy.values,(len(yy),1))) 
    x_train, x_test_val, y_train, y_test_val = train_test_split(xx, yy, shuffle=True,test_size=0.4, random_state=1)
    x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, shuffle=True,test_size=0.25, random_state=1)
    y2 = yy.ravel()
    ind_train = [np.argwhere(y2 == y_train[i])[0].item() for i in range(len(y_train)) if y_train[i] in y2]
    ind_test = [np.argwhere(y2 == y_test[i])[0].item() for i in range(len(y_test)) if y_test[i] in y2]
    ind_val = [np.argwhere(y2 == y_val[i])[0].item() for i in range(len(y_val)) if y_val[i] in y2]
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    y_val = y_val.ravel()
    return x_train, x_test,x_val, y_train, y_test,y_val, xx, yy, np.asarray(ind_train), np.asarray(ind_test), np.asarray(ind_val)

def mean_absolute_percentage_error(y, y_pred) :
    
    if np.where(y==0)[0] != np.nan:
        ind = np.where(y==0)[0]
        y = np.delete(y, ind)
        y_pred = np.delete(y_pred, ind)
    MAPE = 100*np.average(abs(y_pred)/abs(y))
    return MAPE

In [ ]:
df = pd.read_csv('data_6.csv', index_col=None, engine='c') 
print('original datasize:', df.shape)
df.dropna(inplace=True)
print('adjusted datasize:', df.shape)
print('data:')
print(df.head())

In [ ]:
yy=df.iloc[:,-1]# target - kd
xx= df.iloc[:,1:-1] # predictors
x_train, x_test,x_val, y_train, y_test,y_val, x_scaled, y_scaled, ind_train, ind_test,ind_val = split(xx, yy)


print('target is ', yy.name)
print('predictors are ', xx.columns.tolist())
print('samples size is ', xx.shape[0], '; number of predictors is ', xx.shape[1])
print('training data size is ', x_train.shape[0], '; test data size is ', x_test.shape[0],'; validation data size is ', x_val.shape[0])

In [ ]:
gamm=1 / (22 * x_train.var()) # gamma setting
gamm_pusu=gamm
# print(gamm)
print("***Traditional support vector machine***")
print('hyperparameters: C=', 1.0, 'gamma=', gamm, 'epsilon=', 0.1)
SVM_model = SVR(kernel='rbf',C=1.0, gamma=gamm,epsilon=0.1)
SVM_model.fit(x_train, y_train)
y_pred_train=SVM_model.predict(x_train)
y_pred_test=SVM_model.predict(x_test)
y_pred_val=SVM_model.predict(x_val)

y_pred_train = gpr.predict(x_train)    
y_pred_test = gpr.predict(x_test)    
y_pred_val = gpr.predict(x_val) 

print('SVM training R2:{:.2f} rmse:{:.3f} mae:{:.3f}'.format(gpr.score(x_train,y_train), mean_squared_error(y_train, y_pred_train),
                                                        mean_absolute_percentage_error(y_train, y_pred_train)))
print('SVM test R2:{:.2f} rmse:{:.3f} mae:{:.3f}'.format(gpr.score(x_test,y_test), mean_squared_error(y_test, y_pred_test),
                                                        mean_absolute_percentage_error(y_test, y_pred_test)))
print('SVM valid R2:{:.2f} rmse:{:.3f} mae:{:.3f}'.format(gpr.score(x_val,y_val), mean_squared_error(y_val, y_pred_val),
                                                       mean_absolute_percentage_error(y_val, y_pred_val)))

In [ ]:
# Genetic algorithm - Support vector machine
def SVM_model(C,gamma,epsilon,x_train,x_test,y_train):
    
    model = SVR(kernel='rbf',C=C, gamma=gamma,epsilon=epsilon)
    model.fit(x_train, y_train)
    y_pred_test=model.predict(x_test)
    y_pred_train=model.predict(x_train)
    
    return model, y_pred_train, y_pred_test


def GA_SVM(x_train,x_test,y_train, y_test,basename='prediction'):
    
    gamm=1 / (22 * x_train.var()) # gamma setting
    gamm_pusu=gamm
    EPOCHS=20

    # for ES's evaluation
    def svm_score1(C, gamma,epsilon):
        model = SVR(C=C, gamma=gamma,epsilon=epsilon)
        model.fit(x_train, y_train.ravel())
        # print(model.score(x_test, y_test))
        return model.score(x_test, y_test.ravel())
    
    p = ES(svm_score1,param_len=3,size=10,x_min=[1, gamm_pusu,0.005], x_max=[10, 1,0.15],alpha=0.1) #svm_score
    
    C, gamma,epsilon = p.solve(epoch=EPOCHS,verbose=False)

    print('BEST hyperparameters: C=', C, 'gamma=', gamma,'epsilon=',epsilon)
    GA_model, y_pred_train, y_pred_test = SVM_model(C,gamma,epsilon,x_train,x_test, y_train)
    
    return  GA_model


ga_svm = GA_SVM(x_train_noise,x_test,y_train,y_test)
y_pred_train = ga_svm.predict(x_train)    
y_pred_test = ga_svm.predict(x_test)    
y_pred_val = ga_svm.predict(x_val)  
print('GA-SVM training R2:{:.2f} rmse:{:.3f} mae:{:.3f}'.format(ga_svm.score(x_train,y_train), mean_squared_error(y_train, y_pred_train),
                                                        mean_absolute_error(y_train, y_pred_train)))
print('GA-SVM test R2:{:.2f} rmse:{:.3f} mae:{:.3f}'.format(ga_svm.score(x_test,y_test), mean_squared_error(y_test, y_pred_test),
                                                        mean_absolute_error(y_test, y_pred_test)))
print('GA-SVM test R2:{:.2f} rmse:{:.3f} mae:{:.3f}'.format(ga_svm.score(x_val,y_val), mean_squared_error(y_val, y_pred_val),
                                                        mean_absolute_error(y_val, y_pred_val)))